In [123]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Beispielhafte Daten
# X = np.random.rand(100, 13)
# y = np.random.rand(100, 5)

# Deine echten Daten hier laden
X = ... # Dein Input-Datensatz (100, 13)
y = ... # Dein Output-Datensatz (100, 5)

In [124]:
data = pd.read_csv("initial_data.csv")

#print(data)
#print(data.keys())

x = data[['Engine speed', 'Engine load', 'Railpressure', 'Air supply', 'Crank angle', 'Intake pressure', 'Back pressure', 'Intake temperature']]
y = data[['NOx', 'PM 1', 'CO2', 'PM 2', 'Pressure cylinder']]

x_max = x.max()
x_min = x.min()
y_max = y.max()
y_min = y.min()

#print(str(y_min) + "\n:\n" + str(y_max))

y_range = y_max - y_min

print(y_range)

NOx                  685.846823
PM 1                   5.547569
CO2                  160.188892
PM 2                  15.561156
Pressure cylinder    115.101073
dtype: float64


In [125]:
# Datenaufteilung in Training und Test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)


# Datenstandardisierung
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)



# Umwandlung in PyTorch Tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [126]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(8, 16)
        self.layer2 = nn.Linear(16, 12)
        self.layer3 = nn.Linear(12, 10)
        self.output = nn.Linear(10, 5)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = torch.matmul(x, torch.diag(torch.tensor([1, 1, 1, 1, 1, 1, 1, 1])).to(torch.float32))
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.output(x)
        return x

model = NeuralNet()

In [127]:
# Verlustfunktion und Optimierer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Trainingsparameter
num_epochs = 300
batch_size = 3

# Daten in Batches aufteilen
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [128]:
for epoch in range(num_epochs):
    for i, (inputs, targets) in enumerate(train_loader):
        # Forward-Pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward-Pass und Optimierung
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Modell speichern (optional)
torch.save(model.state_dict(), 'model.pth')

Epoch [10/300], Loss: 0.1790
Epoch [20/300], Loss: 0.2265
Epoch [30/300], Loss: 0.2192
Epoch [40/300], Loss: 0.0476
Epoch [50/300], Loss: 0.0921
Epoch [60/300], Loss: 0.1676
Epoch [70/300], Loss: 0.0619
Epoch [80/300], Loss: 0.0525
Epoch [90/300], Loss: 0.0983
Epoch [100/300], Loss: 0.2203
Epoch [110/300], Loss: 0.1992
Epoch [120/300], Loss: 0.1003
Epoch [130/300], Loss: 0.0352
Epoch [140/300], Loss: 0.0613
Epoch [150/300], Loss: 0.1012
Epoch [160/300], Loss: 0.0479
Epoch [170/300], Loss: 0.0170
Epoch [180/300], Loss: 0.0750
Epoch [190/300], Loss: 0.0296
Epoch [200/300], Loss: 0.0792
Epoch [210/300], Loss: 0.2629
Epoch [220/300], Loss: 0.0520
Epoch [230/300], Loss: 0.0382
Epoch [240/300], Loss: 0.0336
Epoch [250/300], Loss: 0.3545
Epoch [260/300], Loss: 0.0783
Epoch [270/300], Loss: 0.0210
Epoch [280/300], Loss: 0.0760
Epoch [290/300], Loss: 0.0418
Epoch [300/300], Loss: 0.0369


In [129]:
model.eval()  # Setzt das Modell in den Evaluationsmodus
with torch.no_grad():  # Keine Gradientenberechnung
    y_pred = model(X_test)
    #test_loss = criterion(y_pred, y_test)
    #print(y_test)
    
    #print(f'Test Loss: {test_loss.item():.4f}')
    y_pred = scaler_y.inverse_transform(y_pred)
    y_test = scaler_y.inverse_transform(y_test)
    for pred, test in zip(y_pred, y_test):
        print(str(pred) + ":" + str(test))

[50.16998742  0.29901642 20.39050843  1.45022837 55.83422986]:[45.11063706  0.37043239 13.33908759  1.34438471 50.75279046]
[137.13124414   3.12919883  64.87213747  10.57881731  71.97648359]:[144.31417996   3.2442979   63.07336518   9.38207516  71.47676054]
[67.49732175  2.06018466 40.57895232  8.11364189 61.44214902]:[66.86942417  4.09588526 36.29146793 16.28325513 60.49792953]
[62.83617101  2.34888773 36.50451453 10.30166379 59.62885575]:[94.23299035  2.83292106 38.50904745 10.53630092 59.95670593]
[43.37890152  1.08424429 21.93206025  5.70665367 54.94434136]:[41.90699086  0.76024322 22.40993205  4.24715129 54.66871374]
[90.58328518  2.75059232 45.51759878 11.34183166 64.36504142]:[84.29989805  3.41757813 48.34347086 14.36933964 66.03415315]
[565.56219963   1.1377963   76.43880999   2.83326371 107.19086693]:[465.21158656   0.99647182  76.01429871   2.44630606 107.39016145]
[205.88386307   3.22973889  98.91874429   5.78571763 101.63465466]:[165.12383548   3.63098576 104.66582744   7.3

In [130]:
# build matrix of elementwise mean squared error

from sklearn.metrics import mean_squared_error


matrix = []
for pred, test in zip(y_pred, y_test):
    data_set_mse = []
    for p, t in zip(pred, test):
        #print(str(pred) + ":" + str(test))
        mse = mean_squared_error([p], [t])
        data_set_mse.append(mse)
    #print(data_set_mse)

    percentage_to_range = data_set_mse / y_max
    matrix.append(percentage_to_range)
#print(matrix)


In [131]:
# check mse per dataset row

from statistics import mean 

mean_for_predicted_dataset = []

for predicted_dataset in matrix:
    mean_for_predicted_dataset.append(mean(predicted_dataset))

print("\n")
for entry in mean_for_predicted_dataset:
    print(entry)

print("Mean:")
print(mean(mean_for_predicted_dataset))



0.10083614281642696
0.03699725601479078
0.9889789518378173
0.29407614353646955
0.03083851891390508
0.15268627733902634
2.864388656409348
0.556181512307474
0.07103829130675303
0.1368690631075893
Mean:
0.52328908135896


In [132]:
# Ceck Accuracy among features of test data

#print(y_pred.shape)

y_pred_t = np.transpose(y_pred)
y_test_t = np.transpose(y_test)

# print(y_pred_t.shape)
# print(y_test_t.shape)
featurewise_mean = []
for feature_t, feature_p in zip(y_test_t, y_pred_t):
    # print(feature_t)
    # print(feature_p)
    # print(mean_squared_error(feature_t, feature_p))
    featurewise_mean.append(mean_squared_error(feature_t, feature_p))

print(featurewise_mean)

#print(y_range)
featurewise_mean_percentage = featurewise_mean / y_max**2
print("mse per feature in percentage to value range")
print(featurewise_mean_percentage)

[1315.0126996006234, 0.5263929657956787, 17.955746335336666, 8.31620497108278, 5.580669124125534]
mse per feature in percentage to value range
NOx                  0.002655
PM 1                 0.016209
CO2                  0.000675
PM 2                 0.031365
Pressure cylinder    0.000218
dtype: float64


In [135]:
x = torch.tensor([16.4712565906533, 37.609734587256995, 900.1427871055062, 228.42045796100746, -8.592515890505748, 1981.8799610478413, 3509.5803914574226, 48.709871706251796])
testResults = model(x)
print(testResults)
#[125.10746278858385, 0.4371585551795759, 20.70223941458411, 3.4443235692265173, 102.99879114216813]

tensor([ 497.9112,  680.5526,  845.7199,  -16.8309, 1187.9828],
       grad_fn=<ViewBackward0>)
